# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [259]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm
import time

BATCH_SIZE = 32
NUM_EPOCHS = 4

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [260]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [261]:
lr = 0.001
momentum = 0.4
SEED = 1336
np.random.seed(SEED)
torch.manual_seed(SEED)

In [262]:
class SimpleNet(nn.Module):
# TODO:define model

# the data shape is (B, C, H, W)
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=0),
                        nn.ReLU(),
                        nn.MaxPool2d(2,2),
                        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=0),
                        nn.ReLU(),
                        nn.MaxPool2d(2,1),
                        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
                        nn.ReLU(),
                        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=0),
                        nn.ReLU())
        
        self.layer2 = nn.Sequential(
                        nn.Linear(32*6*6, 8*8*4),
                        nn.Dropout(p=0.15),
                        nn.ReLU(),
                        nn.Linear(8*8*4, 64),
                        nn.Dropout(p=0.15),
                        nn.ReLU(),
                        nn.Linear(64,10),
                        nn.Softmax())
    
    def forward(self, x):
        x2 = self.layer1(x)
        x2 = x2.reshape(x2.shape[0],-1)
        y = self.layer2(x2)
        return y


model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

Next, we can start to train and evaluate!

In [263]:
def eval_model(model, loader):
    model.eval()
    correct = 0
    allsize = 0
    with torch.no_grad():
        for images, labels in loader:
            output = model(images)
            predict_labels = torch.argmax(output, dim=1)
            correct += (predict_labels.detach().numpy() == labels.detach().numpy()).sum()
            allsize += labels.shape[0]
    return correct / allsize

In [264]:
# train and evaluate
loss_list = []
for epoch in range(NUM_EPOCHS):
    model.train()
    i = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        output = model(images)
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        if (i % 200 == 0):
            loss_list.append(loss.detach().numpy())
    print('In epoch{}, losses are:'.format(epoch), np.array(loss_list))
    loss_list = []
    #evaluate
    #TODO:calculate the accuracy using traning and testing dataset
    if (epoch % 3 == 0):
        train_accu = eval_model(model, train_loader)
        test_accu =  eval_model(model, test_loader)
        print('epoch {}: train_acc = {}, test_acc = {}'.format(epoch, train_accu, test_accu))


100%|██████████| 1875/1875 [01:01<00:00, 30.28it/s]


In epoch0, losses are: [1.6951088 1.5760039 1.6018698 1.571511  1.5551203 1.5193343 1.4931254
 1.5549539 1.513372 ]


  0%|          | 1/1875 [00:00<04:55,  6.34it/s]

epoch 0: train_acc = 0.9534833333333333, test_acc = 0.9574318910256411


  0%|          | 2/1875 [00:00<01:56, 16.08it/s]

In epoch1, losses are: [1.5021613 1.5232528 1.517288  1.4924052 1.5520636 1.4923773 1.5304203
 1.4958313 1.4925917]


  0%|          | 2/1875 [00:00<02:31, 12.39it/s]

In epoch2, losses are: [1.4924418 1.4924197 1.523873  1.4611506 1.4611503 1.4924003 1.4611502
 1.4934151 1.4923986]


100%|██████████| 1875/1875 [02:39<00:00, 11.73it/s]


In epoch3, losses are: [1.5236505 1.4611502 1.4611539 1.4685715 1.5549359 1.4924003 1.5851537
 1.5236605 1.5549037]
epoch 3: train_acc = 0.9692333333333333, test_acc = 0.9714543269230769


#### Q5:
Please print the training and testing accuracy.

In [265]:
train_accu = eval_model(model, train_loader)
test_accu =  eval_model(model, test_loader)
print('epoch {}: train_acc = {}, test_acc = {}'.format(epoch, train_accu, test_accu))

epoch 3: train_acc = 0.9692333333333333, test_acc = 0.9714543269230769
